In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader




ModuleNotFoundError: No module named 'transformers'

## Theoretical considerations to Use-case
- Is this an RAG/document answering or a summarization task?
- If a summarization task, encoder-decoder-style models are typically used in generative tasks where the output heavily relies on the input.
- For text-generation tasks, 

In [2]:

llm = Ollama(
    model="llama2",
    repeat_last_n=10,
    temperature=0.1,
    repeat_penalty = 1.0,
    top_k= 20,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
                                      
            

In [13]:
"""
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>
{{ user_message }} [/INST]
"""

'\n<s>[INST] <<SYS>>\n{{ system_prompt }}\n<</SYS>>\n{{ user_message }} [/INST]\n'

In [3]:
B_INST, E_INST = "<s>[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

loader = PyPDFLoader("/Users/udaykeith/Downloads/ge_2023_Q3_10q.pdf")
pages = loader.load_and_split()

example_description = """
Daktronics, Inc. (DAKT) is publicly traded manufacturer of electronic scoreboards, programmable display systems and large screen video displays for sporting, commercial, and transportation applications.  DAKT does business worldwide and is headquartered in Brookings, SD.  The company was originally founded in 1968 by Drs. Aelred Kurtenback and Duane Sander.  Today the company employs about 2,500 people globally.  
 
DAKT's five business segments include:  Commercial, Live Events, High School Park and Rec, Transportation, and International. """
DEFAULT_SYSTEM_PROMPT = """\
Your task is to provide a summary of a company, given only its financial statement.
You will not provide any additional notes or disclaimers and limit the summary to only two paragraphs, nothing more. You will only use information given in the financial statement. Do repeat yourself or even refer to yourself.
"""
instruction = "Given the financials statements the forward slashes, write a summary for the company General Electric //{pages[4]}//"

SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS + instruction

def get_prompt(instruction):
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template
prompt_1 = get_prompt(SYSTEM_PROMPT)

In [4]:
ge = pages[4].page_content

In [5]:
p1 =  """Write a concise summary of the main ideas in article below in bullet-points. Don't repeat ideas. article: {BODY}""".format(BODY=ge)

In [60]:
help(llm)

Help on Ollama in module langchain_community.llms.ollama object:

class Ollama(langchain_core.language_models.llms.BaseLLM, _OllamaCommon)
 |  Ollama(*, name: Optional[str] = None, base_url: str = 'http://localhost:11434', model: str = 'llama2', mirostat: Optional[int] = None, mirostat_eta: Optional[float] = None, mirostat_tau: Optional[float] = None, num_ctx: Optional[int] = None, num_gpu: Optional[int] = None, num_thread: Optional[int] = None, repeat_last_n: Optional[int] = None, repeat_penalty: Optional[float] = None, temperature: Optional[float] = None, stop: Optional[List[str]] = None, tfs_z: Optional[float] = None, top_k: Optional[int] = None, top_p: Optional[int] = None, system: Optional[str] = None, template: Optional[str] = None, format: Optional[str] = None, timeout: Optional[int] = None, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = N

In [ ]:
Your task is to generate a short summary of a product \
review of an application that is available on playstore.
 
Summarize the review below, delimited by triple 
backticks, in at most 30 words. 
 
Review: ```{prod_review}```

In [87]:
p4 = """<s>[INST] <<SYS>>
You are an underwriter that is reviewing a company. At this point in your analysis, your analyses focues on the financial risk of the company.
<</SYS>>
You task is to write a summary of the company based only on the artcle below delmited by triple backticks. Limit the summary to a maximum of 250 words and a minumum of 
200 words. You must refer to the recent financials in your summary. article: ```{BODY}```
""".format(BODY=ge)

In [69]:
template = """You are an underwriter that is reviewing a company. You task is to provide a summary of the company
based only on the context below delmited by triple backticks, in at most 220 words however using a minumum of 200 words. You must refer to the recent financials in your summary.

context : ```ABOUT GENERAL ELECTRIC.  General Electric Company (General Electric, GE or the Company) is a high-tech industrial \ncompany that operates worldwide through its three segments, Aerospace, Renewable Energy, and Power. Our products include \ncommercial and defense aircraft engines and systems; wind and other renewable energy generation equipment and grid solutions; and \ngas, steam, nuclear and other power generation equipment. We have significant global installed bases of equipment across these \nsectors, and services to support these products are also an important part of our business alongside new equipment sales. \nWe previously announced a strategic plan to form three industry-leading, global, investment-grade public companies from (i) our \nAerospace business, which we plan to refer to as GE Aerospace, (ii) our portfolio of energy businesses, including our Renewable \nEnergy and Power businesses, which we plan to combine and refer to as GE Vernova, and (iii) our former HealthCare business. For \npurposes of this report, we refer to our reporting segments as Aerospace, Renewable Energy and Power. The composition of these \nreporting segments is unchanged. On January 3, 2023, we completed the separation of the HealthCare business from GE through the \nspin-off of GE HealthCare Technologies Inc. (GE HealthCare). See Notes 2 and 3 for further information. The historical results of GE \nHealthCare and certain assets and liabilities included in the spin-off are now reported in GE's consolidated financial statements as \ndiscontinued operations. Additionally, on January 1, 2023, we adopted Accounting Standards Update No. 2018-12, Financial Services – \nInsurance (Topic 944): Targeted Improvements to the Accounting for Long-Duration Contracts . See Note 13 for further information. \nGE’s Internet address at www.ge.com, Investor Relations website at www.ge.com/investor-relations and our corporate blog at \nwww.gereports.com, as well as GE’s LinkedIn and other social media accounts, including @GE_Reports, contain a significant amount \nof information about GE, including financial and other information for investors. GE encourages investors to visit these websites from \ntime to time, as information is updated and new information is posted.  \nMANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF \nOPERATIONS (MD&A). The consolidated financial statements of General Electric Company are prepared in conformity with U.S. \ngenerally accepted accounting principles (GAAP). Unless otherwise noted, tables are presented in U.S. dollars in millions. Certain \ncolumns and rows within tables may not add due to the use of rounded numbers. Percentages presented in this report are calculated \nfrom the underlying numbers in millions. Discussions throughout this MD&A are based on continuing operations unless otherwise noted. \nThe MD&A should be read in conjunction with the Financial Statements and Notes to the consolidated financial statements. \nIn the accompanying analysis of financial information, we sometimes use information derived from consolidated financial data but not \npresented in our financial statements prepared in accordance with GAAP. Certain of these data are considered “non-GAAP financial \nmeasures” under SEC rules. See the Non-GAAP Financial Measures section for the reasons we use these non-GAAP financial \nmeasures and the reconciliations to their most directly comparable GAAP financial measures.\nCONSOLIDATED RESULTS  \nTHIRD  QUARTER 2023  RESULTS. Total revenues were $17.3 billion , up $2.9 billion  for the quarter, driven primarily by increases at \nAerospace and Renewable Energy.\nContinuing earnings (loss) per share was $0.08 . Excluding the results from our run-off Insurance business, gains (losses) on retained \nand sold ownership interests, non-operating benefit costs, interest and other financial charges, separation co

Summary: 
"""

In [62]:
prompt = PromptTemplate(
    input_variables=["context"],
    template=template,
)

In [10]:
llm(p1)

Here are the main ideas from the article:

1. General Electric Company (GE) is a high-tech industrial company that operates globally through its three segments: Aerospace, Renewable Energy, and Power.
2. GE's products include commercial and defense aircraft engines and systems, wind and other renewable energy generation equipment, and gas, steam, nuclear, and other power generation equipment.
3. GE has significant global installed bases of equipment across these sectors, and services to support these products are also an important part of its business.
4. GE has announced a strategic plan to form three industry-leading, global, investment-grade public companies from its Aerospace, energy, and HealthCare businesses.
5. The composition of GE's reporting segments is unchanged, and the historical results of GE HealthCare are now reported in GE's consolidated financial statements as discontinued operations.
6. GE's Internet address, Investor Relations website, and corporate blog contain a s

'Here are the main ideas from the article:\n\n1. General Electric Company (GE) is a high-tech industrial company that operates globally through its three segments: Aerospace, Renewable Energy, and Power.\n2. GE\'s products include commercial and defense aircraft engines and systems, wind and other renewable energy generation equipment, and gas, steam, nuclear, and other power generation equipment.\n3. GE has significant global installed bases of equipment across these sectors, and services to support these products are also an important part of its business.\n4. GE has announced a strategic plan to form three industry-leading, global, investment-grade public companies from its Aerospace, energy, and HealthCare businesses.\n5. The composition of GE\'s reporting segments is unchanged, and the historical results of GE HealthCare are now reported in GE\'s consolidated financial statements as discontinued operations.\n6. GE\'s Internet address, Investor Relations website, and corporate blog

In [71]:
len(response.split(" "))

292

In [70]:
response = (llm(prompt.format(
         query = "")
     )
     )

General Electric Company (GE) is a high-tech industrial company that operates globally across three segments: Aerospace, Renewable Energy, and Power. The company has a significant installed base of equipment across these sectors, and services to support these products, as well as new equipment sales. GE has announced a strategic plan to form three industry-leading, global, investment-grade public companies from its Aerospace, energy, and former HealthCare businesses. The company's historical results of GE HealthCare and certain assets and liabilities included in the spin-off are now reported in GE's consolidated financial statements as discontinued operations. GE has adopted Accounting Standards Update No. 2018-12, Financial Services – Insurance (Topic 944): Targeted Improvements to the Accounting for Long-Duration Contracts. The company's consolidated financial statements are prepared in conformity with US GAAP, and certain columns and rows within tables may not add due to the use of 

In [18]:
llm(prompt.format(text=pages[4].page_content))

KeyError: 'pages'

In [71]:
llm("This is perfect. For future summaries, follow this style exactly.")


Thank you for your feedback! I'm glad you find the summary helpful. To ensure consistency in future summaries, I will follow this style exactly as you have requested. Please let me know if there's anything else I can do for you.

"\nThank you for your feedback! I'm glad you find the summary helpful. To ensure consistency in future summaries, I will follow this style exactly as you have requested. Please let me know if there's anything else I can do for you."

In [72]:
llm("Now write a summary for the company Microsoft.")


Microsoft is a multinational technology company that develops, manufactures, licenses, and supports a wide range of software products, services, and devices. The company was founded in 1975 by Bill Gates and Paul Allen, and is headquartered in Redmond, Washington. Microsoft is one of the largest and most successful technology companies in the world, with a diverse portfolio of products and services that include:

* Operating systems: Microsoft produces the Windows operating system, which is used by millions of computers around the world. The company also offers other operating systems, such as Windows Server and Windows Embedded.
* Productivity software: Microsoft is known for its productivity software, including Word, Excel, and PowerPoint. These applications are used by millions of people around the world for work, school, and personal projects.
* Gaming: Microsoft owns Xbox, a popular gaming platform that includes the Xbox console, Xbox games, and Xbox Live online gaming service.
*

'\nMicrosoft is a multinational technology company that develops, manufactures, licenses, and supports a wide range of software products, services, and devices. The company was founded in 1975 by Bill Gates and Paul Allen, and is headquartered in Redmond, Washington. Microsoft is one of the largest and most successful technology companies in the world, with a diverse portfolio of products and services that include:\n\n* Operating systems: Microsoft produces the Windows operating system, which is used by millions of computers around the world. The company also offers other operating systems, such as Windows Server and Windows Embedded.\n* Productivity software: Microsoft is known for its productivity software, including Word, Excel, and PowerPoint. These applications are used by millions of people around the world for work, school, and personal projects.\n* Gaming: Microsoft owns Xbox, a popular gaming platform that includes the Xbox console, Xbox games, and Xbox Live online gaming serv

In [73]:
llm("This is too long, please follow the example you provided for General Electric.")


My apologies, here is the shortened version of the analysis for General Electric:

GE is a multinational conglomerate that operates in a wide range of industries, including aviation, healthcare, and energy. The company has a long history of innovation and has been at the forefront of several technological advancements. GE has a strong brand and is known for its commitment to quality and reliability.

However, the company has faced several challenges in recent years, including declining sales in some of its core businesses and increased competition from smaller, nimbler competitors. GE has also faced difficulties in its financial services arm, which has resulted in significant losses.

Despite these challenges, GE has a diverse range of businesses and a strong cash flow position, which provides it with the ability to invest in new technologies and opportunities. The company has also been working to simplify its structure and reduce costs, which could help improve its profitability.

Ov

'\nMy apologies, here is the shortened version of the analysis for General Electric:\n\nGE is a multinational conglomerate that operates in a wide range of industries, including aviation, healthcare, and energy. The company has a long history of innovation and has been at the forefront of several technological advancements. GE has a strong brand and is known for its commitment to quality and reliability.\n\nHowever, the company has faced several challenges in recent years, including declining sales in some of its core businesses and increased competition from smaller, nimbler competitors. GE has also faced difficulties in its financial services arm, which has resulted in significant losses.\n\nDespite these challenges, GE has a diverse range of businesses and a strong cash flow position, which provides it with the ability to invest in new technologies and opportunities. The company has also been working to simplify its structure and reduce costs, which could help improve its profitabil

In [7]:
!pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 1.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 1.8 MB/s eta 0:00:0000:0100:01


In [8]:
import weaviate

In [9]:
from weaviate.embedded import EmbeddedOptions